# Importing library 

In [6]:
import pandas as pd
from sqlalchemy import create_engine # used to connect to the database

In [46]:
def Extract_Data():
    
# create function for extract output transformation from transform folder and read 'OLAP' database from SQL Server 
    driver = 'ODBC Driver 17 for SQL Server' # Use the ODBC Driver 17 to connect to a SQL Server database.
    server = 'DESKTOP-I05NNIJ' # Name of service
    database = 'BQ_Snowflake_Modele' # Name of Database 
    
# Create connection string
    conn_str = f"mssql+pyodbc://@{server}/{database}?driver={driver.replace(' ', '+')}&trusted_connection=yes"
    engine = create_engine(conn_str) # 

# Extract Dimension and fact table from SQL Server 
    df_client_Dimension = pd.read_sql("SELECT * FROM Dimension_client", engine)
    df_Date_Dimension = pd.read_sql("SELECT * FROM Dimension_date", engine)
    df_pays_Dimension = pd.read_sql("SELECT * FROM Dimension_pays", engine)
    df_transaction_Dimension = pd.read_sql("SELECT * FROM Dimension_transaction", engine)
    df_villes_Dimension = pd.read_sql("SELECT * FROM Dimension_villes", engine)
    df_Comptes_BQ_Dimension = pd.read_sql("SELECT * FROM Dimension_Comptes_BQ", engine)
    df_fait_transaction = pd.read_sql("SELECT * FROM fait_transaction", engine)


# Extract transform data from output/transfom folder 
    df_clean_clients = pd.read_csv(r'C:/Users/AzComputer/Desktop/ETL(Project)/ETL_Script/Transform/output/df_clean_clients.csv')
    df_clean_villes =  pd.read_csv(r'C:/Users/AzComputer/Desktop/ETL(Project)/ETL_Script/Transform/output/df_clean_villes.csv')
    df_clean_pays =  pd.read_csv(r'C:/Users/AzComputer/Desktop/ETL(Project)/ETL_Script/Transform/output/df_clean_pays.csv')
    df_clean_dates =  pd.read_csv(r'C:/Users/AzComputer/Desktop/ETL(Project)/ETL_Script/Transform/output/df_clean_dates.csv')
    df_clean_compts =  pd.read_csv(r'C:/Users/AzComputer/Desktop/ETL(Project)/ETL_Script/Transform/output/df_clean_compts.csv')
    df_clean_transactions =  pd.read_csv(r'C:/Users/AzComputer/Desktop/ETL(Project)/ETL_Script/Transform/output/df_clean_transactions.csv')

# create a Data  dictionary that return all Dimension and Clean dataframe 
    Data =  {
    "client_Dimension" : df_client_Dimension,
    "Date_Dimension " : df_Date_Dimension,
    "pays_Dimension" : df_pays_Dimension,
    "transaction_Dimension" : df_transaction_Dimension,
    "villes_Dimension" : df_villes_Dimension,
    "Comptes_BQ_Dimension" :  df_Comptes_BQ_Dimension,
    "fait_transaction" : df_fait_transaction,
    "clean_clients" : df_clean_clients,
    "clean_villes" : df_clean_villes,
    "clean_pays" :  df_clean_pays,
    "clean_dates" : df_clean_dates,
    "clean_compts" : df_clean_compts,
    "clean_transactions" : df_clean_transactions
    }
    return Data, engine
    
    
    

In [47]:
Data , engine = Extract_Data() # Call to the function white two variables Data and engine  

In [48]:
df_clean_clients = Data["clean_clients"] # access to the clean Dataframe from the data dictionary

In [49]:
df_clean_villes = Data["clean_villes"]

In [50]:
df_clean_comptes = Data["clean_compts"]

In [51]:
df_clean_dates = Data["clean_dates"]

In [52]:
df_clean_transaction = Data["clean_transactions"]

In [10]:
# Load cleaned data into Dimension table
Data["clean_pays"].to_sql("Dimension_pays", con=engine, if_exists="append", index=False)

46

In [39]:
transaction_columns_Dimension = df_clean_transaction[["transaction_id","account_id","date_id","transaction_type"]]
transaction_columns_Dimension # define juste the speceficific columns for transaction_dimension 

,transaction_id,account_id,date_id,transaction_type
0,13738,2469,3013,retrait
1,48242,2081,1033,retrait
2,93517,188,1708,prélèvement
3,74104,3659,3760,prélèvement
4,44663,8,2686,paiement
...,...,...,...,...
9995,52605,993,3758,virement
9996,99187,3560,283,retrait
9997,31567,2265,3241,dépôt
9998,19415,2122,1416,dépôt


In [35]:
transaction_columns_Dimension.to_sql("Dimension_transaction", con=engine, if_exists="append", index=False)

44

In [23]:
Data["clean_villes"].to_sql("Dimension_villes", con=engine, if_exists="append", index=False)

100

In [25]:
Data["clean_clients"].to_sql("Dimension_client", con=engine, if_exists="append", index=False) 

C:\Users\AzComputer\AppData\Local\Temp\ipykernel_15084\3921175743.py:1: UserWarning: The provided table name 'Dimension_client' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  Data["clean_clients"].to_sql("Dimension_client", con=engine, if_exists="append", index=False)


101

In [13]:
Data["clean_compts"].to_sql("Dimension_Comptes_BQ", con=engine, if_exists="append", index=False)

332

In [23]:
Data["clean_dates"].to_sql("Dimension_date", con=engine, if_exists="append", index=False)

247

In [54]:
ID_client_Fact_table =  df_clean_clients['client_id']      # Dertermine each id from dimensions for loading into a fact table
ID_account_Fact_table = df_clean_comptes['account_id']
ID_transaction_Fact_table = df_clean_transaction['transaction_id']
ID_date_Fact_table =  df_clean_dates['date_id']
Amount_fact_table =  df_clean_transaction['amount']

In [59]:
df_fait_transaction = pd.DataFrame({                     # creates a new pandas DataFrame called df_fait_transaction 
    'client_id': ID_client_Fact_table,
    'account_id': ID_account_Fact_table,
    'transaction_id': ID_transaction_Fact_table,
    'date_id': ID_date_Fact_table,
    'amount': Amount_fact_table
})


In [60]:
df_fait_transaction.to_sql("fait_transaction", con=engine, if_exists="append", index=False) 


363